# Rhythmic distributions

Date: September 29th, 2020.
Author: Benjamin LeBrun.

In this notebook, we assess goodness-of-fit to the regional distributions of rhythmic bigrams. To do so, we first look at goodness of fit $p$-values obtained from a semi-parametric bootstrap (see `bootstrap.py`). Then, for each fit, if $p > 0.1$, we compare it to all other alternative distributions for the region above the corresponding $\hat{x}_{min}$.

In [58]:
import pandas as pd
import numpy as np
import os
import powerlaw as pl
from IPython.utils import io
from tqdm.notebook import tqdm

In [59]:
from distributions import Powerlaw, Exponential, Lognormal, Stretched_exponential, Powerlaw_with_cutoff

In [65]:
data_source = '../data/rhythm/'
boot_source = 'bootstrap/rhythm/'

## 1. Goodness-of-fit $p$-values and parameter estimates

In [169]:
# map names to corresponding distribution objects
dist_dict = {'powerlaw': Powerlaw, 'exponential': Exponential, 
             'lognormal': Lognormal, 'stretched_exponential': Stretched_exponential,
            'truncated_power_law': Powerlaw_with_cutoff}

In [170]:
p_values = pd.DataFrame()
x_mins = pd.DataFrame()
parameters = pd.DataFrame()

regions = ['global','africa', 'asia', 'europe', 'oceania', 'north_america', 'south_america', 
                   'middle_america_and_the_caribbean', 'middle_east']
for dist in tqdm(['powerlaw', 'exponential', 'lognormal', 'stretched_exponential', 'truncated_power_law']):
    dist_ps, dist_xmins, dist_params = [], [], []
    for region in regions:
        try:
            bootstrap = np.array(pd.read_csv(boot_source+'rhythm_'+dist+'_'+region+'.csv').D)
            distribution = dist_dict[dist]
            data = pd.read_csv(data_source+'rhythm_ranks_'+region+'.csv')
            fit = distribution(data.ranks)
            KS = fit.D
            p = len(bootstrap[bootstrap >= KS])/len(bootstrap)
            dist_ps.append(p)
            dist_xmins.append(fit.xmin)
            dist_params.append(fit.get_parameters())
        except Exception as e:
            # it's normal to get missing file errors as some of these region/distribution
            # combinations have not been bootstrapped since the power law is a valid fit
            dist_ps.append(np.nan)
            dist_xmins.append(np.nan)
            dist_params.append(np.nan)
            print(dist, region)
    p_values[dist] = dist_ps
    x_mins[dist] = dist_xmins
    parameters[dist]= dist_params
p_values.index = regions
x_mins.index = regions
parameters.index = regions

Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


exponential asia
exponential europe
exponential oceania
exponential north_america
exponential south_america
lognormal asia
lognormal europe
lognormal oceania
lognormal north_america
lognormal south_america
stretched_exponential asia
stretched_exponential europe
stretched_exponential oceania
stretched_exponential north_america
stretched_exponential south_america
truncated_power_law asia
truncated_power_law europe
truncated_power_law oceania
truncated_power_law north_america
truncated_power_law south_america



Recall that we only fit a given distribution to alternative models when the power-law is not a valid fit. Therefore, there will be `NaN` values in the place of alternative models when the power law is a valid fit. These values should be the ones corresponding to the distributions listed above (i.e. the ones without a bootstrap file).

In [171]:
p_values

,powerlaw,exponential,lognormal,stretched_exponential,truncated_power_law
global,0.000,0.809,0.002,0.857,0.141667
africa,0.064,0.988,0.018,0.004,0.020000
asia,0.116,NaN,NaN,NaN,NaN
europe,0.744,NaN,NaN,NaN,NaN
oceania,0.203,NaN,NaN,NaN,NaN
north_america,0.783,NaN,NaN,NaN,NaN
south_america,0.257,NaN,NaN,NaN,NaN
middle_america_and_the_caribbean,0.017,0.941,0.972,0.981,0.800000
middle_east,0.296,0.000,0.000,0.489,0.000000


In [172]:
x_mins

,powerlaw,exponential,lognormal,stretched_exponential,truncated_power_law
global,7.0,27.0,10.0,31.0,22.0
africa,15.0,69.0,5.0,5.0,9.0
asia,6.0,NaN,NaN,NaN,NaN
europe,10.0,NaN,NaN,NaN,NaN
oceania,16.0,NaN,NaN,NaN,NaN
north_america,9.0,NaN,NaN,NaN,NaN
south_america,8.0,NaN,NaN,NaN,NaN
middle_america_and_the_caribbean,3.0,16.0,12.0,10.0,17.0
middle_east,12.0,1.0,1.0,6.0,1.0


In [173]:
parameters

,powerlaw,exponential,lognormal,stretched_exponential,truncated_power_law
global,[2.5966830010114212],[0.05501348858737702],"[1.5865235416760273, 1.0479954337062993]","[0.03249199356916668, 1.4041128758404728]","[1.0000467812358518, 0.038568299121796615]"
africa,[3.093742637135077],[0.5108156028368804],"[1.1852698905540526, 1.0855722485182242]","[0.9437476441537507, 0.47661682019825713]","[1.4453904734265293, 0.044224853191061564]"
asia,[2.6940089742260422],NaN,NaN,NaN,NaN
europe,[3.9731023774352137],NaN,NaN,NaN,NaN
oceania,[3.286627373274041],NaN,NaN,NaN,NaN
north_america,[4.288603905685319],NaN,NaN,NaN,NaN
south_america,[3.0100464987601554],NaN,NaN,NaN,NaN
middle_america_and_the_caribbean,[2.2951056789202893],[0.1555254424778766],"[2.721710217109792, 0.40722019423283384]","[0.07298913647937622, 1.6016608432618316]","[1.0000253829627024, 0.12235578841157982]"
middle_east,[11.709524201380239],[0.68945064691152],"[0.054797076222348334, 0.9168265476565333]","[0.12634998129607466, 2.587448698899732]","[1.0000372068763537, 0.33262916150826893]"


-----
#### A note on the `rhythm_ranks_middle_east.csv` powerlaw fit
Our estimate for $x_{min}$ in the case of the power law fit to the middle east accounts for only $0.06\%$ of data. This is clearly not a fit we would consider valid. Therefore, we select as estimate for $x_{min}$ the next lowest valud of $D$, which, here, is $9$.

In [85]:
data = np.array(pd.read_csv(data_source+'rhythm_ranks_middle_east.csv').ranks)
len(data[data >= 12])/len(data)*100

0.6032171581769437

In [87]:
Powerlaw(data).Ds

Calculating best minimal value for power law fit


array([0.13929755, 0.11191629, 0.15191631, 0.12958684, 0.13545071,
       0.12964725, 0.12495066, 0.12327752, 0.08995381, 0.1199193 ,
       0.10335906, 0.07854785, 0.16315363, 1.        ])

In [90]:
bootstrap = np.array(pd.read_csv(boot_source+'rhythm_powerlaw_middle_east_xmin9.csv').D)
data = pd.read_csv(data_source+'rhythm_ranks_'+region+'.csv')
fit = Powerlaw(data.ranks, xmin=9)
KS = fit.D
p = len(bootstrap[bootstrap >= KS])/len(bootstrap)
print('p = %.4f' % p)

p = 0.0780


Calculating best minimal value for power law fit


Also, at $x_{min} = 2$.

In [91]:
bootstrap = np.array(pd.read_csv(boot_source+'rhythm_powerlaw_middle_east_xmin2.csv').D)
data = pd.read_csv(data_source+'rhythm_ranks_'+region+'.csv')
fit = Powerlaw(data.ranks, xmin=2)
KS = fit.D
p = len(bootstrap[bootstrap >= KS])/len(bootstrap)
print('p = %.4f' % p)

p = 0.0000


Calculating best minimal value for power law fit


We reject the fit in both cases.

In [96]:
# update the dataframes
p_values.at['middle_east', 'powerlaw'] =  0.078
x_mins.at['middle_east', 'powerlaw'] =  9
parameters.at['middle_east', 'powerlaw'] = Powerlaw(data.ranks, xmin=9).get_parameters()

Calculating best minimal value for power law fit
/Users/benlebrun/opt/anaconda3/lib/python3.7/site-packages/powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))


We have the same issue for `africa` when fitting to the exponential distribution.  

In [132]:
data = np.array(pd.read_csv(data_source+'rhythm_ranks_africa.csv').ranks)
len(data[data >= 69])/len(data)*100

0.04485310607759588

In [134]:
Exponential(data).Ds[0:5]

[[<powerlaw.Fit at 0x1a23456450>, 69.0, 1.2378986724570495e-13],
 [<powerlaw.Fit at 0x1a23456310>, 21.0, 0.03739133018557861],
 [<powerlaw.Fit at 0x1a23456690>, 20.0, 0.037745159356799396],
 [<powerlaw.Fit at 0x1a23456b50>, 22.0, 0.03885945625828868],
 [<powerlaw.Fit at 0x1a23456dd0>, 23.0, 0.04068390942433453]]

In [136]:
bootstrap = np.array(pd.read_csv(boot_source+'rhythm_exponential_africa_xmin21.csv').D)
data = pd.read_csv(data_source+'rhythm_ranks_'+'africa'+'.csv')
fit = Exponential(data.ranks, xmin=21)
KS = fit.D
p = len(bootstrap[bootstrap >= KS])/len(bootstrap)
print('p = %.3f' % p)

p = 0.239


In [137]:
# update the dataframes
p_values.at['africa', 'exponential'] = 0.239
x_mins.at['africa', 'exponential'] =  21
parameters.at['africa', 'exponential'] = Exponential(data.ranks, xmin=21).get_parameters()

In [138]:
Exponential(data.ranks, xmin=21).get_parameters()

[0.07317673102939362]

---
## 2. Alternative distribution comparison

Recall that **each valid fit** must be compared to other fits using the likelihood ratio test. So, for each fit with a $p$-value greater than $0.1$, we compare it to all alternative fits above $\hat{x}_{min}$ using the likelihood ratio test. If the test is significant ($p < 0.05$) and the value of $LR$ is negative, we can reject the fit in question.

In [70]:
def likelihood_ratio_tests(fit, fit_dist):
    distributions = ['power_law', 'lognormal', 'truncated_power_law', 'stretched_exponential', 'exponential']
    results = []
    for dist in distributions:
        if dist == fit_dist:
            results.append((-1,-1))
            continue
        with io.capture_output() as captured:
            try:
                res = fit.distribution_compare(fit_dist, dist)
            except ZeroDivisionError:
                res = (-1,-1)
        results.append(res)
            
    return results

In [71]:
dataframes = {}
for region in tqdm(p_values.index):
    data = pd.read_csv(data_source+'rhythm_ranks_'+region+'.csv')
    df = pd.DataFrame()
    for dist in p_values.columns:
        p_value = p_values.loc[region, dist]
        if p_value > 0.1:
            x_min = x_mins.loc[region, dist]
            if dist == 'powerlaw': 
                dist = 'power_law' # naming convention
            results = likelihood_ratio_tests(pl.Fit(data.ranks, xmin=x_min, discrete=True), dist)
            df[dist] = results
        else:
            df[dist] = [np.nan for i in range(0,5)]
    df.index = ['power_law', 'lognormal', 'truncated_power_law', 'stretched_exponential', 'exponential']
    df = df.dropna(axis=1, how='all')
    dataframes[region] = df

/Users/benlebrun/opt/anaconda3/lib/python3.7/site-packages/powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
/Users/benlebrun/opt/anaconda3/lib/python3.7/site-packages/powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))


We now list the dataframes containing the likelihood ratio results for each region. The only comparisons used are those to the power law.

In [73]:
dataframes['global']

,exponential,stretched_exponential,truncated_power_law
power_law,"(24.627675185666956, 2.183496960652464e-07)","(23.908462077208164, 7.483881437243601e-06)","(24.78908292798087, 1.9062529332813938e-12)"
lognormal,"(2.470673260407804, 0.0013928111761822104)","(2.4229421848294894, 0.0007571349129786556)","(3.7655411024450145, 1.3188544184969003e-06)"
truncated_power_law,"(2.444643829786844, 0.08012499792836397)","(5.081695413455158, 0.05950007687469179)","(-1, -1)"
stretched_exponential,"(-0.5025634964709456, 0.31607309776603865)","(-1, -1)","(0.4494676952404655, 0.6245737190617056)"
exponential,"(-1, -1)","(2.0906066268877206, 0.04087442991798307)","(1.3329784107618092, 0.51402815598214)"


In [74]:
dataframes['africa']

,exponential
power_law,"(-inf, nan)"
lognormal,"(-0.7926874299386261, 0.4408617009782997)"
truncated_power_law,"(-1, -1)"
stretched_exponential,"(-0.9379137845896598, 0.17080913046639068)"
exponential,"(-1, -1)"


In [75]:
dataframes['asia']

,power_law
power_law,"(-1, -1)"
lognormal,"(-6.317155817872379, 0.014903246140920014)"
truncated_power_law,"(-7.5543243828616395, 0.00010149721823637936)"
stretched_exponential,"(-6.80533581562551, 0.01039223295076591)"
exponential,"(15.42830465761434, 0.08203712663451564)"


In [76]:
dataframes['europe']

,power_law
power_law,"(-1, -1)"
lognormal,"(-1.0912101051429364, 0.2814943229493748)"
truncated_power_law,"(-1.3548877766293086, 0.09973502295072556)"
stretched_exponential,"(-1.2761433221001885, 0.26051609042760027)"
exponential,"(-1.0829194065022816, 0.5020662429070204)"


In [77]:
dataframes['oceania']

,power_law
power_law,"(-1, -1)"
lognormal,"(-3.0923471469132275, 0.08144127147663373)"
truncated_power_law,"(-3.7028814052927617, 0.006501527003374696)"
stretched_exponential,"(-3.596904281170017, 0.06652288861628883)"
exponential,"(-3.4546617106084216, 0.14340611703671546)"


In [78]:
dataframes['north_america']

,power_law
power_law,"(-1, -1)"
lognormal,"(-1.4457161807114445, 0.20808652609136258)"
truncated_power_law,"(-1.824640061492699, 0.05609345153392431)"
stretched_exponential,"(-1.6477292543453608, 0.1915856539344123)"
exponential,"(-0.4481780065700529, 0.8593143959769981)"


In [79]:
dataframes['south_america']

,power_law
power_law,"(-1, -1)"
lognormal,"(-4.257662156588082, 0.03925260646300424)"
truncated_power_law,"(-5.62500250094611, 0.0007962280119299114)"
stretched_exponential,"(-4.8036988903523765, 0.030734338553940088)"
exponential,"(7.127483649017502, 0.2644049286437692)"


In [80]:
dataframes['middle_america_and_the_caribbean']

,exponential,lognormal,stretched_exponential,truncated_power_law
power_law,"(4.441109222771949, 0.0048783160935390426)","(16.88140856756464, 0.00019489668843982334)","(26.626170527776274, 1.4678119613158616e-06)","(2.9887885809204304, 0.014488892920699548)"
lognormal,"(-0.04205649458918925, 0.9513165516188873)","(-1, -1)","(0.7045305403741744, 0.478555392079012)","(-0.19281853073527389, 0.8046511395682101)"
truncated_power_law,"(0.47431050395764074, 0.1981792738071444)","(5.000527747116147, 0.0816731565651734)","(8.468652242271697, 0.013649017361785984)","(-1, -1)"
stretched_exponential,"(-0.4117257820479221, 0.36417293852852195)","(-0.18501774087856893, 0.8094536698702548)","(-1, -1)","(-0.5707379333428966, 0.5137168579559803)"
exponential,"(-1, -1)","(2.49379003829533, 0.25015232268531007)","(4.006523980853806, 0.00464415061602752)","(-0.3080041554765698, 0.32094487718972997)"


In [81]:
dataframes['middle_east']

,power_law,stretched_exponential
power_law,"(-1, -1)","(12.284270493379882, 0.0015390191071551524)"
lognormal,"(-0.03456535941157013, 0.8335290119032226)","(0.9751094462000667, 0.1512698089152126)"
truncated_power_law,"(-1, -1)","(6.246358779955285, 0.03442360657953599)"
stretched_exponential,"(-0.037673158447863786, 0.7906380773167774)","(-1, -1)"
exponential,"(-0.036706010445153314, 0.7809467091089781)","(4.787200632317676, 0.001973091900169255)"


In [111]:
def sub_object_map(fit, model):
    if model == 'exponential':
        return fit.exponential
    elif model == 'stretched_exponential':
        return fit.stretched_exponential
    elif model == 'lognormal':
        return fit.stretched_exponential
    elif model == 'truncated_powerlaw':
        return fit.truncated_powerlaw
    elif model == 'powerlaw':
        return fit
    else:
        return -1

In [129]:
def plot_ccdf(region, model, xmin=False):
    distribution = dist_dict[model]
    data = pd.read_csv(data_source+'rhythm_ranks_'+region+'.csv')
    if xmin:
        fit = pl.Fit(data.ranks, discrete=True, xmin=xmin)
    else:
        Fit = distribution(data.ranks)
        fit = Fit.fit
    fig = pl.plot_ccdf(color='r', data=fit.data_original, label=r'Empircal, no $x_{min}$')
    pl.plot_ccdf(color='b', data=fit.data, ax=fig, label=r'Empircal, $x_{min} = %s$' % fit.xmin)
    obj = sub_object_map(fit, model)
    obj.plot_ccdf(color='b', linestyle='--', ax=fig)
    fig.set_ylabel("p(X ≥ x)")
    fig.set_xlabel('rank')
    fig.set_title('Complementary CDF of bigram ranks')
    handles, labels = fig.get_legend_handles_labels()
    leg = fig.legend(handles, labels, loc=3)
    leg.draw_frame(False)